In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from bingX import BingX
import mplfinance as mpl
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
# WIP: Pandas TA LIB
import pandas_ta as ta

TRADE_INTERVALS = ["1M","1d","4h","1h","30m","15m","5m","3m","1m"]
API_KEY = os.environ.get("BINGX_API_KEY","")
SECRET_KEY = os.environ.get("BINGX_SECRET_KEY","")

bingx_client = BingX(API_KEY, SECRET_KEY)

# Date intervals to fetch data
date = datetime(2024,3,31,12,0,0)
date_end = datetime(2024,4,2,12,0,0)
start_time_ms = int(date.timestamp() * 1000)
end_time_ms = int(date_end.timestamp() * 1000)

# Get Perpetual data of <Pair> between time_intervals for each TRADE_INTERVAL
data =[bingx_client.perpetual_v2.market.get_k_line_data(symbol="BTC-USDT",interval=interval,start_time=start_time_ms,end_time=end_time_ms) for interval in TRADE_INTERVALS ]

'''
df = pd.DataFrame(data[0]) # Monthly
df = pd.DataFrame(data[1]) # Daily
df = pd.DataFrame(data[2]) # H4
df = pd.DataFrame(data[3]) # H1
df = pd.DataFrame(data[4]) # M30
df = pd.DataFrame(data[5]) # M15
df = pd.DataFrame(data[6]) # M5
df = pd.DataFrame(data[7]) # M3
df = pd.DataFrame(data[8]) # M1
'''
df = pd.DataFrame(data[3]) # H1

# Cast data to DF types
df['time'] = pd.to_datetime(df['time'],unit='ms')
df = df.astype({'open':'float'})
df = df.astype({'close':'float'})
df = df.astype({'high':'float'})
df = df.astype({'low':'float'})
df = df.astype({'volume':'float'})

# Chart idx must be the timestamp
df = df.set_index('time',inplace=False)

In [ ]:
# Calculate the mid of the candle
df['median'] = np.abs(df['high'] + df['low']) / 2

# Remove High and low as we dont need them
#pdf = lf.drop(columns=['high','low'])

# Number of points for relative extrema
n = 5
# Get max/min of median values using Relative Extrema
df['min'] = df.iloc[argrelextrema(df['median'].values, np.less_equal,
                                  order=n)[0]]['median']
df['max'] = df.iloc[argrelextrema(df['median'].values, np.greater_equal,
                                  order=n)[0]]['median']

# Calculate EMA 10 using TA-LIB
df['ema_10'] = ta.sma(df["close"])

# Drop unused columns
lf = df.drop(columns=['volume','open','close','high','low'])

# Show max/min as points green/red
plt.scatter(lf.index,lf['min'],c='r')
plt.scatter(lf.index,lf['max'],c='g')

# Use the calculated median values
plt.plot(lf.index,lf)

# Showing chart bitches!
plt.show()